In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**TASK-2**

# Moving to the directory where data is stored

In [1]:
cd ../input/task-2

/kaggle/input/task-2


# Importing the required libraries

In [2]:
import pandas as pd
import numpy as np
import tensorflow.keras
from keras.preprocessing.image import ImageDataGenerator

In [3]:
from tensorflow.keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPool2D,AvgPool2D,BatchNormalization,Activation,Input
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.applications import InceptionV3

# Variable learning rate with lr=0.001, deacy= 0.001/200, momnetum=0.9

In [4]:
lr_schedule = tensorflow.keras.optimizers.SGD(
    lr=0.001,
    decay= 0.001/200,
    momentum=0.9)

# Data flow pipeline with addition of some noise (brightness and zoom)

In [16]:
datagen= ImageDataGenerator(
zoom_range=0.15,
brightness_range = [0.3,1.4],
validation_split=0.20)
train_it=datagen.flow_from_directory('train_task2',class_mode='categorical',batch_size=20,target_size=(28,28),shuffle=True,color_mode='grayscale',subset='training')
test_it= datagen.flow_from_directory('train_task2',class_mode='categorical',batch_size=20,target_size=(28,28),shuffle=True,color_mode='grayscale',subset='validation')

Found 320 images belonging to 10 classes.
Found 80 images belonging to 10 classes.


# Model architecture with structure inspired from VGG with addition of dropout and batch normalization layers to avoid overfitting and reduce the training time.

In [17]:
model=Sequential()
model.add(Conv2D(32, (5, 5), padding="same",input_shape=(28,28,1)))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding="same"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(256, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(50))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(10))
model.add(Activation("softmax"))

# Compiling the model

In [18]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer= lr_schedule)


# Saving the weights in the file with max val accuracy 

In [19]:
from keras.callbacks import ModelCheckpoint
filepath='../../working/mymodel2.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


# Training the model

In [23]:
model.fit(train_it,steps_per_epoch=int(320/20),epochs=50,validation_data=test_it,validation_steps=int(80/20),callbacks=callbacks_list)


Epoch 1/50
16/16 [==============================] - 8s 527ms/step - loss: 0.0734 - accuracy: 0.9906 - val_loss: 0.5086 - val_accuracy: 0.8375

Epoch 00001: val_accuracy did not improve from 0.88750
Epoch 2/50
16/16 [==============================] - 8s 505ms/step - loss: 0.0899 - accuracy: 0.9719 - val_loss: 0.4731 - val_accuracy: 0.8625

Epoch 00002: val_accuracy did not improve from 0.88750
Epoch 3/50
16/16 [==============================] - 8s 510ms/step - loss: 0.0858 - accuracy: 0.9844 - val_loss: 0.7216 - val_accuracy: 0.7875

Epoch 00003: val_accuracy did not improve from 0.88750
Epoch 4/50
16/16 [==============================] - 8s 501ms/step - loss: 0.1421 - accuracy: 0.9594 - val_loss: 0.5044 - val_accuracy: 0.8125

Epoch 00004: val_accuracy did not improve from 0.88750
Epoch 5/50
16/16 [==============================] - 8s 504ms/step - loss: 0.0805 - accuracy: 0.9844 - val_loss: 0.5555 - val_accuracy: 0.8000

Epoch 00005: val_accuracy did not improve from 0.88750
Epoch 6/50

# Max val accuracy reached to 90%

# MNIST Data

In [10]:
from keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()
import tensorflow as tf
train_X=train_X.reshape(60000,28,28,1)
test_X= test_X.reshape(10000,28,28,1)

11493376/11490434 [==============================] - 0s 0us/step


# Applying onehotencoding in the output data

In [11]:
from sklearn.preprocessing import OneHotEncoder
test_y=test_y.reshape(10000,1)
# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')
# passing bridge-types-cat column (label encoded values of bridge_types)
test_y = pd.DataFrame(enc.fit_transform(test_y).toarray())
# merge with main df bridge_df on key values

#bridge_df = bridge_df.join(enc_df)

In [12]:
from sklearn.preprocessing import OneHotEncoder
train_y=train_y.reshape(60000,1)
# creating instance of one-hot-encoder
enc = OneHotEncoder(handle_unknown='ignore')
# passing bridge-types-cat column (label encoded values of bridge_types)
train_y = pd.DataFrame(enc.fit_transform(train_y).toarray())


# Same model with random initialized weights

In [13]:
model1=Sequential()
model1.add(Conv2D(32, (5, 5), padding="same",input_shape=(28,28,1)))
model1.add(Activation("relu"))
model1.add(BatchNormalization())
model1.add(Conv2D(32, (3, 3), padding="same"))
model1.add(BatchNormalization())
model1.add(MaxPool2D(pool_size=(2, 2)))
model1.add(Dropout(0.2))
model1.add(Conv2D(64, (3, 3), padding="same"))
model1.add(Activation("relu"))
model1.add(BatchNormalization())
model1.add(Conv2D(64, (3, 3), padding="same"))
model1.add(Activation("relu"))
model1.add(BatchNormalization())
model1.add(MaxPool2D(pool_size=(2, 2)))
model1.add(Dropout(0.2))
model1.add(Conv2D(128, (3, 3), padding="same"))
model1.add(Activation("relu"))
model1.add(BatchNormalization())
model1.add(Conv2D(128, (3, 3), padding="same"))
model1.add(Activation("relu"))
model1.add(BatchNormalization())
model1.add(MaxPool2D(pool_size=(2, 2)))
model1.add(Dropout(0.2))
model1.add(Conv2D(256, (3, 3), padding="same"))
model1.add(Activation("relu"))
model1.add(BatchNormalization())
model1.add(Conv2D(256, (3, 3), padding="same"))
model1.add(Activation("relu"))
model1.add(BatchNormalization())
model1.add(MaxPool2D(pool_size=(2, 2)))
model1.add(Dropout(0.2))
model1.add(Flatten())
model1.add(Dense(50))
model1.add(Activation("relu"))
model1.add(BatchNormalization())
model1.add(Dropout(0.3))
model1.add(Dense(10))
model1.add(Activation("softmax"))

# Compiling and training the model

In [14]:
model1.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer= lr_schedule)
model1.fit(train_X,train_y,batch_size=50,epochs=50,validation_data=(test_X,test_y),callbacks=callbacks_list)

Epoch 1/50
1200/1200 [==============================] - 318s 264ms/step - loss: 0.7408 - accuracy: 0.7812 - val_loss: 0.0531 - val_accuracy: 0.9827
Epoch 2/50
1200/1200 [==============================] - 317s 264ms/step - loss: 0.1190 - accuracy: 0.9676 - val_loss: 0.0391 - val_accuracy: 0.9869
Epoch 3/50
1200/1200 [==============================] - 315s 263ms/step - loss: 0.0804 - accuracy: 0.9781 - val_loss: 0.0318 - val_accuracy: 0.9893
Epoch 4/50
1200/1200 [==============================] - 315s 262ms/step - loss: 0.0672 - accuracy: 0.9815 - val_loss: 0.0292 - val_accuracy: 0.9904
Epoch 5/50
1200/1200 [==============================] - 314s 261ms/step - loss: 0.0552 - accuracy: 0.9852 - val_loss: 0.0256 - val_accuracy: 0.9905
Epoch 6/50
1200/1200 [==============================] - 314s 262ms/step - loss: 0.0486 - accuracy: 0.9866 - val_loss: 0.0229 - val_accuracy: 0.9926
Epoch 7/50
1200/1200 [==============================] - 313s 261ms/step - loss: 0.0439 - accuracy: 0.9874 - val_

# Compiling and training the same model with weights pretrained from dataset given in task-1 (The first trained model of this notebook is used below). Since the val accuracy reached above 99%, I stopped the training after 15 epochs due to which keyboard interrupt error is shown below(due to time limitations of the notebook session on kaggle)

In [15]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer= lr_schedule)
model.fit(train_X,train_y,batch_size=50,epochs=50,validation_data=(test_X,test_y))

Epoch 1/50
1200/1200 [==============================] - 317s 263ms/step - loss: 0.6923 - accuracy: 0.8046 - val_loss: 0.0628 - val_accuracy: 0.9796
Epoch 2/50
1200/1200 [==============================] - 316s 263ms/step - loss: 0.1374 - accuracy: 0.9640 - val_loss: 0.0445 - val_accuracy: 0.9852
Epoch 3/50
1200/1200 [==============================] - 315s 263ms/step - loss: 0.0990 - accuracy: 0.9740 - val_loss: 0.0343 - val_accuracy: 0.9885
Epoch 4/50
1200/1200 [==============================] - 316s 263ms/step - loss: 0.0774 - accuracy: 0.9792 - val_loss: 0.0289 - val_accuracy: 0.9901
Epoch 5/50
1200/1200 [==============================] - 316s 264ms/step - loss: 0.0643 - accuracy: 0.9832 - val_loss: 0.0285 - val_accuracy: 0.9904
Epoch 6/50
1200/1200 [==============================] - 317s 264ms/step - loss: 0.0584 - accuracy: 0.9840 - val_loss: 0.0243 - val_accuracy: 0.9906
Epoch 7/50
1200/1200 [==============================] - 317s 264ms/step - loss: 0.0531 - accuracy: 0.9857 - val_

KeyboardInterrupt: 

#  Both have almost the same convergence time with max val accuracy of above 99%. The pretrained ones accuracy reached upto 99.46% and the one trained from scrach reached upto 99.50% in first 15 epochs.